In [ ]:
# Performance test of recommendation algorithm
# Random customers on general recommendation
# Single thread

In [18]:
import numpy as np
import random
import time
from datetime import datetime

In [3]:
# initialize all data for collaborative

# customer to demographic lookup
#arr_cxd = create_mock_cust_demo(NUM_OF_CUSTOMERS,NUM_OF_DEMOREGION,NUM_OF_DEMOGENDER)
#arr_cxd = np.load('demo_matrix.npy')
arr_cxd = np.load('../../data/extracts/cluster1CustmeridsMapping.npy')
print "created customer to demographic lookup"

# customer to order lookup
#matrix_co = create_mock_cust_orders(NUM_OF_CUSTOMERS,NUM_OF_PRODUCTS,0.01)
matrix_co = np.load('../../data/extracts/cust_item_matrix.npy')

print "created customer to order lookup"

# content rating lookup
arr_conrate = np.load('../../data/derived/rating_indexed.npy')

print "created content lookup"



created customer to demographic lookup
created customer to order lookup
created content lookup


In [4]:
# Create co-occurrence matrix

def create_cocmatrix(subset_matrix_cust_order):
    rows, cols = subset_matrix_cust_order.shape
    m = np.zeros((cols,cols))
    for i in range(cols):
        t = np.sum(subset_matrix_cust_order[subset_matrix_cust_order[:,i] > 0],axis=0)
        t[i] = 0
        m[i,:] = t
        if i % (cols/5) == 0:
            print "created rows in cooccurrence matrix at row",i
    return m

In [5]:
# generate recommendations
# purchase_vec = customer purchases (vector)
# cocm = co occurrence matrix [items x items]
# num_rec = number of recommendations
def gen_recom_collab(purchase_list,cocm,num_rec):
    rowsum = np.zeros(cocm.shape[0])
    
    for p in purchase_list:
        rowsum += cocm[p,:]
        
    rowsum[purchase_list] = 0
    indices = np.nonzero(rowsum)[0]
    toprec = indices[np.argsort(rowsum[indices])][-1 * num_rec:][::-1]
    return list(toprec)

In [6]:
# generate recommendations
# purchase_vec = customer purchases (vector)
# list_rating = content rating
# num_rec = number of recommendations
def gen_recom_content(purchase_list,list_rating,num_rec):
    NUM_RECOMMENDATIONS_C = 40
    rowsum = np.copy(list_rating)
        
    rowsum[purchase_list] = 0
    
    indices = np.nonzero(rowsum)[0]
    randrec = indices[np.argsort(rowsum[indices])][-1 * NUM_RECOMMENDATIONS_C:][::-1]
    randindices = np.random.permutation(randrec)[:num_rec]
    toprec = randindices[np.argsort(rowsum[randindices])][::-1]
    return list(toprec)

In [9]:
list_useful_clusters = [1,3,4,7]

list_coo_matrix = {}
list_coo_matrix[0] = create_cocmatrix(matrix_co)
for c in list_useful_clusters:
    list_sub = list(np.where(arr_cxd == c)[0])
    list_coo_matrix[c] = create_cocmatrix(matrix_co[list_sub])

created rows in cooccurrence matrix at row 0
created rows in cooccurrence matrix at row 798
created rows in cooccurrence matrix at row 1596
created rows in cooccurrence matrix at row 2394
created rows in cooccurrence matrix at row 3192
created rows in cooccurrence matrix at row 0
created rows in cooccurrence matrix at row 798
created rows in cooccurrence matrix at row 1596
created rows in cooccurrence matrix at row 2394
created rows in cooccurrence matrix at row 3192
created rows in cooccurrence matrix at row 0
created rows in cooccurrence matrix at row 798
created rows in cooccurrence matrix at row 1596
created rows in cooccurrence matrix at row 2394
created rows in cooccurrence matrix at row 3192
created rows in cooccurrence matrix at row 0
created rows in cooccurrence matrix at row 798
created rows in cooccurrence matrix at row 1596
created rows in cooccurrence matrix at row 2394
created rows in cooccurrence matrix at row 3192
created rows in cooccurrence matrix at row 0
created row

In [19]:
def random_tester(i,mco,coc,c_rating,important_c,num_rec_collab,num_rec_content):
    time_current = datetime.now()
    p_vec = mco[i,:]
    list_original_purchases = np.where(p_vec > 0)[0]
    
    list_summed_coo_vec = []
    list_content_vec = []
    list_hybrid_vec = []
    
    cust_cluster = coc[i][0]
    
    list_content_vec = gen_recom_content(list_original_purchases,c_rating,num_rec_collab+num_rec_content)
    
    if cust_cluster in important_c:
        list_summed_coo_vec = gen_recom_collab(list_original_purchases,list_coo_matrix[cust_cluster],num_rec_collab+num_rec_content)
    else:
        list_summed_coo_vec = gen_recom_collab(list_original_purchases,list_coo_matrix[0],num_rec_collab+num_rec_content)
    
    list_hybrid_vec = list_summed_coo_vec[:num_rec_collab]
    
    for r in list_content_vec:
        if len(list_hybrid_vec) >= num_rec_collab+num_rec_content:
            break
        if r not in list_hybrid_vec:
            list_hybrid_vec.append(r)
    return ((datetime.now() - time_current).total_seconds() * 1000)

In [40]:
list_times = []
num_requests = 100000

for i in list(np.random.choice(matrix_co.shape[0], num_requests, replace=True)):
    list_times.append(random_tester(i,matrix_co,arr_cxd,arr_conrate,list_coo_matrix,9,1))


In [45]:
# Average recommendation time
print "average recommendation return time " + str(np.mean(list_times)) + " ms"

average recommendation return time 1.91313547 ms


In [46]:
# Recommendations per second (# requests / (total time in seconds))
print "requests per second: " + str(num_requests/(np.sum(list_times)/1000))

requests per second: 522.702137763


0.0